## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

In [1]:
import pandas as pd
bank = pd.read_csv("/datasets/data.csv")
print(bank.info())
print(bank.head(15))

for i in bank.columns:
    print()
    print("В колонке  {} зафиксированы следующие уникальные значения: {}.".format(i, bank[i].unique()))
    print()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None
    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   

In [2]:
display(bank.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

В дата фрейме 21525 столбцов, при этом необходимо отметить следующее: 
1)столбец days_employed имеет 19351 значение (следовательно имеем дело с пропусками) и тип float64 (часть даных со знаком минус, что противоречит понятию "трудового стажа")
2) Столбец total_income также имеет только 19351 значения и тип  float64 
3)В колонке children замечено два нестандартных случая -1 ребенок (знак "-", скорее всего поставлен по ошибке) и 20 детей
4)колонку education необходимо привести к единому образцу
5)В колонке gender вместо пропуска стоит строка 'XNA' 6) возраст ряда заемщиков равен нулю

### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
def income_change(row):
    if row['total_income']==0:
        a=int(grouped_not_null_income.loc[grouped_not_null_income['income_type']==row['income_type'],'total_income'])
        return a
    else:
        return row['total_income'] #функция для замены значений 'total_income' равных нулю
    

# исследуем пропуски на случайность
display(((bank[bank['total_income'].isnull()]['income_type'].value_counts())/(bank['income_type'].value_counts())).fillna(0)*100)

median_age=bank.loc[bank['dob_years'] != 0, 'dob_years'].median() # в этом блоке меняем равные нулю значения 'dob_years'
bank[bank['dob_years']==0]['dob_years']=median_age                # на медианное значения возраста
print(bank.loc[bank['dob_years'] == median_age, ])

bank.loc[bank['gender'] == 'XNA', 'gender'] = None # устраняем неизвестное значение пола заемщика
print(bank.groupby('gender')['gender'].count())

print(bank[bank.days_employed.isnull()])  #проверка корреляции недостающих данных
print(set(bank[bank.days_employed.isnull()].index==bank[bank.total_income.isnull()].index))

bank['total_income']=bank['total_income'].fillna(0) # создаем датафрейм в котором отражена медианный доход по типу занятости
not_null_income=bank[bank['total_income']!=0]
grouped_not_null_income=not_null_income.groupby('income_type')['total_income'].median().reset_index()

bank['total_income']=bank.apply(income_change, axis=1) # заменяем недостающие данные по даходу на медианные значения 






                                                                                        

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

безработный         0.000000
в декрете           0.000000
госслужащий        10.075394
компаньон           9.990167
пенсионер          10.710581
предприниматель    50.000000
сотрудник           9.937944
студент             0.000000
Name: income_type, dtype: float64

       children  days_employed  dob_years            education  education_id  \
6             0   -2879.202052         43               высшее             0   
37            0   -6448.810860         43               ВЫСШЕЕ             0   
43            0   -4375.681384         43  неоконченное высшее             2   
77            0    -726.943771         43               высшее             0   
127           0   -5229.552069         43               ВЫСШЕЕ             0   
...         ...            ...        ...                  ...           ...   
21270         1     -85.282723         43              среднее             1   
21382         1   -4560.953044         43              среднее             1   
21468         0    -785.453500         43               ВЫСШЕЕ             0   
21469         0   -2808.519596         43               высшее             0   
21520         1   -4529.316663         43              среднее             1   

               family_status  family_st

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Вывод

Проведя первичную проверку, мы выяснили, что в колонках days_employed и total_income недостает данных. 
Необходимо выяснить есть ли взаимосвязь между отсутствием данных в этих колонках.
Выведя на печать рассматриваемый датафрейм, мы видим, что отсутствие данных в этих столбцах коррелирует между собой.
Выполним проверку :set(bank[bank.days_employed.isnull()].index==bank[bank.total_income.isnull()].index) => True

В этом блоке мы : 1)изменили нулевые значения возраста на медианные 2)устранили неизвестное значение пола заемщика 3) заменили недостающие значения дохода на медианные в соответствии с типом дохода

Недостающие данные трудового стажа БУДУТ заполнены ПОСЛЕ ЗАМЕНЫ ТИПА ДАННЫХ.явной закономерности по 


* Исследовав пропущенные значения на случайность, мы определили , что большая часть пропущенных значений- предприниматели (видимо, скрывают свои доходы). У части безработных, студентов и дам в декрете данных может не быть по причине отсутствия дохода в рассматриваемый интервал времени.

### Замена типа данных

In [4]:
def employ_days(row):#функция замены недостоверного трудового стажа
    y=row['days_employed']/365
    if (y>(row['dob_years']-20))or (row['days_employed']==0):
        return (row['dob_years']-20)*365
    else:
        return row['days_employed']

bank['total_income']=bank['total_income'].astype(int)# переведем для удобства доход в целочисленную величину
bank['children']=bank['children'].apply(abs) #Бармалей ест детей-исправляем минусовое значение в количестве детей и приводим данные к целочисленным значениям
bank['days_employed']=bank['days_employed'].fillna(0)#заменим недостоющие данные
bank['days_employed']=bank['days_employed'].apply(abs).astype(int)#уберем отрицательные значения

print(bank['days_employed']/365)# видим, что граф Дракула отработал 900 лет 
bank['days_employed']=bank.apply(employ_days, axis=1)




0         23.115068
1         11.024658
2         15.405479
3         11.298630
4        932.235616
            ...    
21520     12.408219
21521    942.293151
21522      5.789041
21523      8.526027
21524      5.435616
Name: days_employed, Length: 21525, dtype: float64


Количество детей и число дней трудового стажа- натуральные числа. Мы преобразовали их в положительный целочисленный вид. (total_income был преобразован для удобства дальнейшей работы)

в этом блоке: 1) были убраны отрицательные значения в столбце 'children' 2) привели к целочисленному значению 'total_income' 3) определили недоставерные значения в столбце 'days_employed' и заменили их и пропущенные значения на максимально возможный стаж.


### Обработка дубликатов

In [5]:
print(bank['education'])
bank['education']=bank['education'].str.lower()# приводим столбец к нижнему регистру

bank.drop_duplicates(subset =['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income'],keep = 'first', inplace = True) # удаляем дубликаты учитывая все столбцы кроме 'purpose'
bank.info()


0         высшее
1        среднее
2        Среднее
3        среднее
4        среднее
          ...   
21520    среднее
21521    среднее
21522    среднее
21523    среднее
21524    среднее
Name: education, Length: 21525, dtype: object
<class 'pandas.core.frame.DataFrame'>
Int64Index: 20777 entries, 0 to 21524
Data columns (total 12 columns):
children            20777 non-null int64
days_employed       20777 non-null int64
dob_years           20777 non-null int64
education           20777 non-null object
education_id        20777 non-null int64
family_status       20777 non-null object
family_status_id    20777 non-null int64
gender              20776 non-null object
income_type         20777 non-null object
debt                20777 non-null int64
total_income        20777 non-null int64
purpose             20777 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Вывод

 в этом блоке мы привели столбец 'education' к единому образцу и удалили дубликаты ( все строки в которых все столбцы кроме 'purpose' совпадают). Столбец 'purpose' не учитывали, так как Лемматизация- следующий шаг.

### Лемматизация

In [6]:
from pymystem3 import Mystem
m = Mystem()

def lem(row):# функция присваивающая категорию цели на основании леммы столбца 'purpose'
    lemma=m.lemmatize(row['purpose'])
    for i in category:
        if i in lemma:
            return i
            break
    
#опишем категории целей взятия ссуды
z=[]
for i in bank['purpose'].unique():
    lemmas = m.lemmatize(i)
    z+=lemmas

words = pd.DataFrame({'words': z})
words=words.loc[(words['words']!=' ')&(words['words']!='\n')]
grouped_words=words.groupby('words')['words'].count()
print(bank['purpose'].unique())
print(grouped_words)

category=['жилье','недвижимость','автомобиль','образование','свадьба']

purporse_frame=bank.apply(lem,axis=1)
bank['purpose']=purporse_frame
bank.drop_duplicates(keep = 'first', inplace = True)# проверка на дубликаты после стандартизации столбца 'purpose'

print(bank)




['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']
words
автомобиль         9
вы

### Вывод

В этом блоке на основании лемматизации уникальных значений столбца 'purpose' мы создали перечень категорий целей взятия ссуды.
Далее, лемматизировав столбец 'purpose', мы присвоили созданные ранее категории.

### Категоризация данных

In [7]:
def age_group(age):
        if age <= 35:
                return 'молодой возраст'
        elif (age>35) and (age<=60):
                return 'средний возраст'
        else:
                return 'пожилой возраст'
            
            
def income_group(income):
        if income <= quantile_35:
                return 'низкий доход'
        elif (income>quantile_35) and (income<=quantile_75):
                return 'средний доход'
        else:
                return 'высокий доход'            
            
            
quantile_35=bank['total_income'].quantile(.35)  
quantile_75=bank['total_income'].quantile(.75)

bank['total_income']=bank['total_income'].apply(income_group)
bank['dob_years']=bank['dob_years'].apply(age_group)

final_data=bank.reset_index(drop=True)

final_data.info()

print(final_data)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20777 entries, 0 to 20776
Data columns (total 12 columns):
children            20777 non-null int64
days_employed       20777 non-null int64
dob_years           20777 non-null object
education           20777 non-null object
education_id        20777 non-null int64
family_status       20777 non-null object
family_status_id    20777 non-null int64
gender              20776 non-null object
income_type         20777 non-null object
debt                20777 non-null int64
total_income        20777 non-null object
purpose             20777 non-null object
dtypes: int64(5), object(7)
memory usage: 1.9+ MB
       children  days_employed        dob_years education  education_id  \
0             1           8030  средний возраст    высшее             0   
1             1           4024  средний возраст   среднее             1   
2             0           4745  молодой возраст   среднее             1   
3             3           4124  молодой во

### Вывод

В этом блоке мы классифицировали наши данные по возрасту( написали функцию, которая заменяет данные в столбце 'dob_years' на трикатегории в соответствии с годами) и и по доходу ( написали функцию, которая которая заменяет данные в столбце 'total_income' на три категории в соответствии с квантилями значений этого столбца)

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [8]:
pivot_children_data=final_data.pivot_table(index=["children"],  aggfunc={'children':'count', 'debt':'sum'})
pivot_children_data.rename(columns={'children': 'number_of_people', 'debt': 'number_of_debtors'}, inplace=True)
pivot_children_data['debt_percent']=(pivot_children_data['number_of_debtors']/pivot_children_data['number_of_people'])*100
pivot_children_data=pivot_children_data.reset_index().round(2)
display(pivot_children_data)



,children,number_of_people,number_of_debtors,debt_percent
0,0,13581,1056,7.78
1,1,4737,444,9.37
2,2,2006,193,9.62
3,3,328,27,8.23
4,4,40,4,10.00
5,5,9,0,0.00
6,20,76,8,10.53


### Вывод

Количество должников без детей превышает все остальные группы по абсолютному значению,однако, рассмотрев относительное значение, мы видим , что многодетным семьям сложнее выплачивать кредиты.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [9]:
pivot_family_data=final_data.pivot_table(index=["family_status"],  aggfunc={"family_status":'count', 'debt':'sum'})
pivot_family_data.rename(columns={"family_status": 'number_of_people', 'debt': 'number_of_debtors'}, inplace=True)
pivot_family_data['debt_percent']=(pivot_family_data['number_of_debtors']/pivot_family_data['number_of_people'])*100
pivot_family_data=pivot_family_data.reset_index().round(2)
display(pivot_family_data)





,family_status,number_of_debtors,number_of_people,debt_percent
0,Не женат / не замужем,274,2755,9.95
1,в разводе,85,1183,7.19
2,вдовец / вдова,63,930,6.77
3,гражданский брак,387,4071,9.51
4,женат / замужем,923,11838,7.80


### Вывод

Проанализировав данные по семейному положению, можно сделать вывод, что большая часть заемщиков ( а следовательно и должников)- это семейные пары (семейный бюджет требует больше затрат). Однако относительные величины показывают, что процент должников больше среди холостых или живущих гражданским браком людей (вероятно, эта группа может позволить себе более легкомысленной отношение к жизни)

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [10]:
pivot_income_data=final_data.pivot_table(index=["total_income"],  aggfunc={"total_income":'count', 'debt':'sum'})
pivot_income_data.rename(columns={"total_income": 'number_of_people', 'debt': 'number_of_debtors'}, inplace=True)
pivot_income_data['debt_percent']=(pivot_income_data['number_of_debtors']/pivot_income_data['number_of_people'])*100
pivot_income_data=pivot_income_data.reset_index().round(2)
display(pivot_income_data)




,total_income,number_of_debtors,number_of_people,debt_percent
0,высокий доход,367,5194,7.07
1,низкий доход,611,7272,8.40
2,средний доход,754,8311,9.07


### Вывод

Среди людей со среднем доходом больше должников, как по абсолютной, так и по относительной велечинам. Предположительно, люди с высоким доходом меньше нуждаются в деньгах, а с низким боятся брать на себя лишние обязательства.

- Как разные цели кредита влияют на его возврат в срок?

In [11]:
pivot_purpose_data=final_data.pivot_table(index=["purpose"],  aggfunc={"purpose":'count', 'debt':'sum'})
pivot_purpose_data.rename(columns={"purpose": 'number_of_people', 'debt': 'number_of_debtors'}, inplace=True)
pivot_purpose_data['debt_percent']=(pivot_purpose_data['number_of_debtors']/pivot_purpose_data['number_of_people'])*100
pivot_income_data=pivot_income_data.reset_index().round(2)
display(pivot_purpose_data)

deep_diging=final_data.pivot_table(index=["purpose"],columns="children"  ,aggfunc={"debt":(lambda x: ((x.sum()/x.count())*100))}).round(2)
display(deep_diging)




,number_of_debtors,number_of_people,debt_percent
purpose,,,
автомобиль,400,4173,9.585430
жилье,307,4315,7.114716
недвижимость,472,6148,7.677293
образование,367,3853,9.525045
свадьба,186,2288,8.129371


debt                                        
children        0      1      2      3      4    5      20
purpose                                                   
автомобиль    8.80  10.91  11.99   8.33  10.00  0.0  12.50
жилье         6.84   7.18   8.61   8.22  18.18  0.0   0.00
недвижимость  7.01   9.13   8.74   7.37  11.11  0.0   9.52
образование   8.99  10.50  11.83   5.88   0.00  0.0   6.67
свадьба       7.67   9.60   5.66  15.62   0.00  0.0  33.33

### Вывод

Самые выделяющиеся категории трат это ссуда на автомобиль и образование. Вероятно, автомобиль в кредит берет более инфантильная группа населения, а кредит на образование берут молодые люди, не имеющие финансовой подушки безопасности.
В свою очередь, заемы на жилье и недвижимость отдают чаще ( может быть, это следствие продуманности решения).


* мы распределили имеющуюся информацию по целям взятия кредита и количеству детей. Анализ показал, что рекордсменами по задолжности ( 33%) являются люди, которые ,имея 20 детей, решили отыграть шикарную свадьбу. 

### Шаг 4. Общий вывод

Рассмотрев процент должников относительно различных категорий, можно сделать вывод, что самые надежные заемщики - семейные люди с высоким доходом, берущие ссуду на недвижимость. В свою очередь, многодетные семьи или люди, берущие кредит на воплощение своей мечты ( машина, свадьба. образование) попадают в зону риска. Необходимо отметить, что ярко выделяющихся неблагонадежных категорий не замечено ( отличия всего в пару процентов), и для принятия решения нужен индивидуальный анализ каждого клиента. 

